<a href="https://colab.research.google.com/github/YeisonHunt/RL_algorithm_tesis/blob/master/mecanismoMLRedWifi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#===============================================================#
# LIBRERIAS
import numpy as np
from numpy.random.mtrand import random_integers

#print("== Libraries OK! ==")

In [ ]:
#===============================================================#
#                         MLActuator                            #
#===============================================================#
class MLActuator:

  #=== Constructor
  def __init__(self, dicStates, dicActions):

    #Estados
    self.dicStates = dicStates
    self.old_state = "00"
    self.state = "00"

    #Acciones
    self.dicActions = dicActions
    self.lastAction = "00"

    print("MLActuator Created!")

  def get_lastAction(self):
    return self.lastAction

  def get_state_key(self):
    return self.state

  def get_old_state_key(self):
    return self.old_state

  def get_old_state_value(self):
    if self.old_state in self.dicStates:
      return self.dicStates[self.old_state]
    else:
      return "None"

  #=== Step: Ejecuta la acción
  def step(self, action):

    print("==")
    print("Actuator Step")

    if action in self.dicActions:

      #Actualizar estado anterior
      self.old_state = self.state

      #Obtener estado según la acción
      self.state = action

      #print("XXX old_state: " + self.dicStates[self.old_state])

      #### AQUI LLAMAR FUNCIONES DE TRASPASO (CODIGO ANDRES) ###

      #Mostrar estado anterior
      if self.old_state in self.dicStates:
        print("old_state: " + self.dicStates[self.old_state])

      #Mostrar estado nuevo
      print("state: " + self.dicStates[self.state])

      #Guardar última acción ejecutada
      self.lastAction = action

    else:
      print("Acción no valida!!!")

    print("End Actuator Step!")
    print("")

#print("== MLActuator OK! ==")

In [ ]:
#===============================================================#
#                           MLAgent                             #
#===============================================================#

class MLAgent:

  #=== Constructor
  def __init__(self, dicStates, dicActions, discount_factor, learning_rate, ratio_explotacion):
    #Variables definidas por desarrollador
    self.dicStates = dicStates
    self.dicActions = dicActions

    self.nStates = len(dicStates)
    self.nActions = len(dicActions)

    #Ecuación de Bellman
    self.discount_factor = discount_factor  #L
    self.learning_rate = learning_rate      #a
    self.ratio_explotacion = ratio_explotacion

    #POLITICAS (Q-TABLE)
    self.q_table = np.zeros([self.nStates, self.nActions])

    print("MLAgent Created!")

  #==Sección Bellman
  def setQParameters(self, discount_factor, learning_rate, ratio_explotacion):
    self.discount_factor = discount_factor  #L
    self.learning_rate = learning_rate      #a
    self.ratio_explotacion = ratio_explotacion

  #=== Método para tomar el siguiente paso
  def get_next_step(self, state, nAccessPoint, nSlices):
    
    #Variable para determinar si el aprendizaje se completa
    q_table_completed = True

    print("==")
    print("Agent Step")

    #Elegir si explotar o explorar para decidir si seguir aprendiendo o elegir la mejor opción
    explotar = np.random.uniform()
    if explotar<=self.ratio_explotacion:
      elegir_mejor_opcion = False
      print("Explotar    : " + str(explotar) + "<=" + str(self.ratio_explotacion))
    else:
      elegir_mejor_opcion = True
      print("NO Explotar : " + str(explotar) + ">" + str(self.ratio_explotacion))

    #NOTA: Pero si existe un mejor paso entonces dar el mejor paso (Mejor acción)
    posible_next_stepi = 0
    if elegir_mejor_opcion:
      maxQ = -1
      posible_next_stepi = -1
      statei = list(self.dicActions.keys()).index(state)

      for ai in range(0, self.nActions):
        q_value = self.q_table[int(statei),ai]
        if(q_value>maxQ):
          maxQ = q_value
          posible_next_stepi = int(ai)

      if(posible_next_stepi>=0):
        posible_next_step = list(self.dicActions.keys())[posible_next_stepi]
        #next_step = posible_next_step;
        print("Mejor Acción (Posible): " + self.dicActions[posible_next_step])
    else:
      ################################################
      #Dar un paso aleatorio (Acción aleatoria): 
      #- Definido por el desarrollo: Conectarse a Slice_jk Aleatoriamente
      posible_next_stepi = random_integers(0, self.nActions-1)
      posible_next_step = list(self.dicActions.keys())[posible_next_stepi]
      print("Acción Aleatoria (Posible): " + self.dicActions[posible_next_step])
      ################################################

    next_step = posible_next_step

    #Mostrar paso siguiente seleccionado
    print("Acción Seleccionada : " + self.dicActions[next_step])
    print("End Agent Step!")
    print("==")

    return next_step

  #=== Método para actualizar las políticas con las recompensas obtenidas
  def update(self, old_state, action_taken, reward_action_taken):

    print("==")
    print("Agent Update!")

    if old_state in self.dicStates:
      print("oldstate: " + self.dicStates[old_state])
    
    if action_taken in self.dicActions:
      print("actiontk: " + self.dicActions[action_taken])

    #obtener índice correspondiente al estado en la Q-Table
    idx_old_state = int(list(self.dicStates.keys()).index(old_state))

    #obtener índice correspondiente de la acción en la Q-Table
    idx_action_taken = int(list(self.dicActions.keys()).index(action_taken))

    #### AQUI ACTUALIZAR Q-TABLE ###
    ### Aplicar ecuación de Bellman ###

    #Obtener parámetros de la ecuación de Bellman
    Qsa = self.q_table[idx_old_state,idx_action_taken] #Q(S,A)

    L = self.discount_factor     #Tasa de descuento
    a = self.learning_rate       #Radio de aprendizaje
    R = reward_action_taken #Recompensa

    #maxQ = max(self.q_table[idx_old_state:]).any()
    #print("MaxQ =" + str(maxQ))

    maxQ = 0
    for ai in range(0,self.nActions):
      q_value = self.q_table[idx_old_state, ai]
      if(q_value>maxQ):
        maxQ = q_value

    print("MaxQ =" + str(maxQ))

    self.q_table[idx_old_state,idx_action_taken] = Qsa + a*(R+(L*maxQ)-Qsa)

    print("End Agent Update!")
    print("==")


  #=== Método para mostrar la tabla de políticas (Q-Table)
  def showQ(self):
    print("Q-TABLE")
    print(self.q_table)
    print("#===============================================================#")

#print("== MLAgent OK! ==")


In [ ]:
#===============================================================#
#                      MechanismBasedOnML                       #
#===============================================================#
class MechanismBasedOnML:

  #=== Constructor
  def __init__(self, nAccessPoint, nSlices, discount_factor, learning_rate, ratio_explotacion):

    # Datos de la red WiFi
    self.nAccessPoint = nAccessPoint
    self.nSlices = nSlices
    
    # Posibles Estados Estático
    self.dicStates = {"00":"NO conectado"}
    for ai in range(1,self.nAccessPoint+1):
      for si in range(0,self.nSlices):
        #key = str(ai) + str(si)  #FORMATO STRING
        key = ai*10 + si          #FORMATO INT
        self.dicStates.update({key :"Conectado("+ str(ai) + "," + str(si)+")"})

    # Posibles Acciones Estático
    self.dicActions = {"00":"Desconectarse"}
    for ai in range(1,self.nAccessPoint+1):
      for si in range(0,self.nSlices):
        #key = str(ai) + str(si)  #FORMATO STRING
        key = ai*10 + si          #FORMATO INT
        self.dicActions.update({key :"Conectar_a("+ str(ai) + "," + str(si)+")"})

    #Agente
    self.agent = MLAgent(self.dicStates, self.dicActions, discount_factor, learning_rate, ratio_explotacion)

    #Actuador
    self.actuator = MLActuator(self.dicStates, self.dicActions)
    
    #Recompensa/Penalización
    self.reward = 0

    #Variables auxiliares
    self.old_state = "00"

    print("MechanismBasedOnML Created!")
    
  #=== Step: Ejecuta la acción
  def step(self):
    self.old_state = self.actuator.get_state_key()
    new_action = self.agent.get_next_step(self.old_state, self.nAccessPoint, self.nSlices)
    self.actuator.step(new_action)

  #=== Update: Actualiza las políticas
  def update(self, new_action, reward):
    self.agent.update(self.old_state, new_action, reward)
    self.agent.showQ()

#print("== MechanismBasedOnML OK! ==")



In [ ]:
#===============================================================#
#                        SDNController                          #
#===============================================================#
class SDNController:

  #=== Constructor
  def __init__(self):
    self.state = "00"
    self.reward = 0
    print("SDNController Created!")


  #=== Asignar estado
  def set_State(self, newState):
    self.state = newState

  #=== Calcular recompensa
  def calcularReward(self):
    # Calcular reward según las variables
    self.reward = 1

  #=== Monitoreo
  def monitoring(self, j_k_list, action):

    #Procesar aquí los parámetros para obtener el nuevo estado y la recompensa

    #Validar si el estado=acción está en la lista de Slice que el carro escucha
    if action in j_k_list.keys():

      print()
      print("j_k_list " + str(action))
      print(j_k_list[action])
      print("================")    


      #== Orden de datos sel Slice_j_k
      # slice_id
      # target_mac_j_k
      # latency_server
      # load_j_k_t_1
      # load_j_k_t
      # rssi_j_k_car_id_t_1
      # rssi_j_k_car_id_t

      #########################################
      #slice_id = list(j_k_list.keys())[idx]
      #########################################

      slice_id = action 

      target_mac_j_k = j_k_list[slice_id][0] 
      latency_server = j_k_list[slice_id][1]
      load_j_k_t_1 = j_k_list[slice_id][2]
      load_j_k_t = j_k_list[slice_id][3]
      rssi_j_k_car_id_t_1 = j_k_list[slice_id][4]
      rssi_j_k_car_id_t = j_k_list[slice_id][5]

      #Mostrar datos del Slice
      print("slice_id: " + slice_id)
      print("target_mac_j_k: " + target_mac_j_k)
      print("latency_server: " + str(latency_server))
      print("load_j_k_t_1: " + str(load_j_k_t_1))
      print("load_j_k_t: " + str(load_j_k_t))
      print("rssi_j_k_car_id_t_1: " + str(rssi_j_k_car_id_t_1))
      print("rssi_j_k_car_id_t: " + str(rssi_j_k_car_id_t))

      #=========================================
      #== CALCULO DE RECOMPENSAS
      print("=========================================")
      print("RECOMPENSAS:")
      print("")

      #== CALCULAR VARIACIÓN DE CARROS
      E = 0.5
      slice_variacionCarros_j_k = load_j_k_t - load_j_k_t_1
      print("VARIACION DE CARROS: " + str(slice_variacionCarros_j_k))

      #== CALCULAR LATENCIA DE TRASPASO (HandoverLatency)
      C = 0.5
      HandoverLatency_Car = 1
      print("HANDOVER LATENCY: " + str(HandoverLatency_Car))
      
      

      #== CALCULAR RECOMPENSA TOTAL
      self.reward = E * slice_variacionCarros_j_k - C * HandoverLatency_Car
      print("")
      print("RECOMPENSA TOTAL: " + str(self.reward))
      #=========================================

      #Obtener nuevo estado según la acción
      self.state = action
      
      #Mostrar estado nuevo
      print("=========================================")
      print("NUEVO ESTADO:" + self.state)
      print("")

      print("=========================================")
    
    else:
      print("=========================================")
      print("Acción no escuchada")
      print("=========================================")


    return self.state, self.reward



#print("== SDNController OK! ==")

